In [35]:
using Pkg
for p in ["Knet", "Plots", "IterTools","PyCall"]
    if !haskey(Pkg.installed(),p)
        Pkg.add(p);
    end
end
using DelimitedFiles
using Knet: KnetArray, accuracy, progress, minibatch, cycle, adam, xavier_uniform
using Plots
using IterTools: ncycle, takenth, take
using Base.Iterators: flatten
using LinearAlgebra

In [36]:
include("utils.jl")
include("models.jl")

┌ Warning: `getindex(o::PyObject, s::AbstractString)` is deprecated in favor of dot overloading (`getproperty`) so elements should now be accessed as e.g. `o."s"` instead of `o["s"]`.
│   caller = top-level scope at none:0
└ @ Core none:0


In [37]:
adj, features, labels, idx_train, idx_val, idx_test = load_dataset("nell")

(
  [19982,     1]  =  1
  [41507,     2]  =  1
  [9215 ,     3]  =  1
  [18079,     4]  =  1
  [18081,     4]  =  1
  [21930,     4]  =  1
  [9742 ,     5]  =  1
  [42496,     5]  =  1
  [42504,     5]  =  1
  [42505,     5]  =  1
  [8929 ,     6]  =  1
  [8946 ,     6]  =  1
  ⋮
  [8945 , 65751]  =  1
  [8929 , 65752]  =  1
  [8937 , 65752]  =  1
  [8961 , 65753]  =  1
  [8964 , 65753]  =  1
  [9875 , 65753]  =  1
  [8975 , 65754]  =  1
  [9045 , 65754]  =  1
  [9105 , 65754]  =  1
  [8937 , 65755]  =  1
  [8950 , 65755]  =  1
  [8985 , 65755]  =  1, 
  [1    ,     1]  =  1
  [2    ,     1]  =  1
  [3    ,     1]  =  1
  [6    ,     1]  =  2
  [15   ,     1]  =  1
  [27   ,     1]  =  2
  [30   ,     1]  =  3
  [38   ,     1]  =  1
  [46   ,     1]  =  2
  [59   ,     1]  =  1
  [68   ,     1]  =  2
  [72   ,     1]  =  2
  ⋮
  [61267, 65744]  =  1
  [61268, 65745]  =  1
  [61269, 65746]  =  1
  [61270, 65747]  =  1
  [61271, 65748]  =  1
  [61272, 65749]  =  1
  [61273, 65750]  =  1

In [1]:
# TODO: take user inputs
struct args
    epochs
    lr
    weight_decay
    hidden
    pdrop
end

arguments = args(200, 0.01, 1e-5, 64, 0.1)

# Model and optimizer
model = GCN(size(features,1),
            arguments.hidden,
            size(labels,2),
            adj,
            arguments.pdrop)

function val_loss(g::GCN,x,y)
    output = g(x)[:, idx_val]
    nll(output, y[idx_val]) + (arguments.weight_decay * sum(g.layer1.w .* g.layer1.w)) 
end  
function val_loss(g::GCN, d)
    mean(val_loss(g,x,y) for (x,y) in d)
end

function test_loss(g::GCN,x,y)
    output = g(x)[:, idx_test]
    nll(output, y[idx_test]) + (arguments.weight_decay * sum(g.layer1.w .* g.layer1.w)) 
end  
function test_loss(g::GCN,d)
    mean(test_loss(g,x,y) for (x,y) in d)
end

(g::GCN)(x,y) = nll(g(x)[:, idx_train], y[idx_train]) + (arguments.weight_decay * sum(g.layer1.w .* g.layer1.w)) 
#(g::GCN)(x,y) = nll(g(x)[:, idx_train], y[idx_train])

UndefVarError: UndefVarError: features not defined

In [39]:
#output = model(features)

In [40]:
labels_decoded = mapslices(argmax, labels ,dims=2)[:]
labels_decoded = convert(Array{Int64,1}, ones(size(labels_decoded)))

65755-element Array{Int64,1}:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 ⋮
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1

In [41]:
#nll(output[:,idx_train], labels_decoded[idx_train])

In [42]:
#accuracy(output[:,idx_train], labels_decoded[idx_train])

In [43]:
#using AutoGrad
#J = @diff model(features, labels_decoded)
#grad(J, params(model)[1])

In [45]:
data =  minibatch(features, labels_decoded[:], length(labels_decoded[:]))

Data{Tuple{SparseMatrixCSC{Int32,Int64},Array{Int64,1}}}(Int32[1 1 … 0 0; 1 4 … 0 0; … ; 0 0 … 1 0; 0 0 … 0 1], [1 1 … 1 1], 65755, 65755, false, 1, 1:65755, false, (61278, 65755), (65755,), SparseMatrixCSC{Int32,Int64}, Array{Int64,1})

In [46]:
function train_with_results(model, data, epoch, lr)
    training = adam(model, ncycle(data, epoch), lr=lr)
    snapshot() = model(data)
    snapshots = (snapshot() for x in takenth(progress(training),length(data)))
    res = collect(flatten(snapshots))
    return res
end

train_with_results (generic function with 1 method)

In [ ]:
results = train_with_results(model, data, 2, arguments.lr)
trnloss = Array{Float32}(results)
plot(trnloss, ylim=(0.0,2.0),labels=[:trnloss],xlabel="Epochs",ylabel="Loss")

In [ ]:
output = model(features)
accuracy(output[:,idx_train], labels_decoded[idx_train])

In [ ]:
output = model(features)
accuracy(output[:,idx_test], labels_decoded[idx_test])